### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import astropy.units as u
from survey_tools import catalog, match
from survey_tools.utility import plot, table

### Options

In [ ]:
field = 'AEGIS' # AEGIS, COSMOS, GOODS-N, GOODS-S, UDS

do_save = False
do_save_ascii = False
keep_open = False

plot_mode   = 'default'
plot_options   = plot.get_plot_colours(plot_mode)

### Load 3D-HST (v4.1, 2014-09-03)

In [ ]:
catalog_params_3DHST = catalog.get_params('3D-HST', field, 'F160W')
catalog_3DHST = catalog.read(catalog_params_3DHST, force_tables=True)
catalog_region_3DHST = catalog.get_catalog_region(catalog_3DHST, added_distance=0.5*u.degree)
print(f"3D-HST: {catalog_3DHST.count} sources")
print('\n3D-HST Region:')
print(catalog_region_3DHST)

### Cross-Match zCOSMOS Bright (DR3, 2016-01-19)

In [ ]:
if field == 'COSMOS':
    catalog_params_ZCB = catalog.get_params('ZCOSMOS-BRIGHT')
    catalog_ZCB = catalog.read(catalog_params_ZCB)
    print(f"{catalog_ZCB.catalog}: {catalog_ZCB.count} sources")

In [ ]:
if field == 'COSMOS':
    matches_ZCB = match.cross_match_catalogs_by_ra_dec(catalog_3DHST, catalog_ZCB)
    print(f"Number of Matches: {matches_ZCB.count}")

In [ ]:
if field == 'COSMOS':
    match.append_cross_matched_data(matches_ZCB, catalog_3DHST, catalog_ZCB)

In [ ]:
if field == 'COSMOS' and matches_ZCB.count > 0:
    value_filter       = (catalog_3DHST.idmap['id']  < 1e7) & (catalog_3DHST.idmap['ZCB'] >= 0)
    other_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['ZCB'] >= 0) & (catalog_3DHST.idmap['ZCB_dist']  > 0.0)
    multi_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['ZCB'] >= 0) & (catalog_3DHST.idmap['ZCB_dist'] == 0.0) & (catalog_3DHST.idmap['ZCB_num'] > 1)
    dup_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['ZCB'] >= 0) & (catalog_3DHST.idmap['ZCB_dist'] == 0.0) & (catalog_3DHST.idmap['ZCB_num'] == 1)
    non_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['ZCB'] >= 0) & (catalog_3DHST.idmap['ZCB_dist'] == 0.0) & (catalog_3DHST.idmap['ZCB_num'] == 0)

    num_matches = np.sum(value_filter)
    num_other_matches = np.sum(other_match_filter)
    num_multi_matches = np.sum(multi_match_filter)
    num_dup_matches = np.sum(dup_match_filter)
    num_non_matches = np.sum(non_match_filter)
    print(f"Number of {catalog_ZCB.name} Redshifts:")
    print(f"    Matches      : {num_matches} (diff={matches_ZCB.count - num_matches - num_other_matches})")
    print(f"  + Other matches: {num_other_matches}")
    print(f"  + Dup-matches  : {num_dup_matches}")
    print(f"  + Multi-matches: {num_multi_matches}")
    print(f"  + Non-matches  : {num_non_matches}")
    print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_ZCB.count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

    zlim_deep = [0, 3]
    zlim = [0, 2]

    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_ZCB'] > 0.0) & (catalog_3DHST.redshift['z_ZCB_flag'] < 8)
    fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_ZCB.catalog} vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_ZCB'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel(f"$z_{{peak}}$ [{catalog_3DHST.catalog}]")
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_ZCB.catalog}]")
    ax.set_xlim(zlim_deep)
    ax.set_ylim(zlim_deep)

    del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_ZCB' in locals():
    catalog.close(catalog_ZCB)
    del catalog_ZCB, catalog_params_ZCB

### Cross-Match zCOSMOS Deep (unreleased, between 2009-2016?)

In [ ]:
if field == 'COSMOS':
    catalog_params_ZCD = catalog.get_params('ZCOSMOS-DEEP')
    catalog_ZCD = catalog.read(catalog_params_ZCD)
    print(f"{catalog_ZCD.catalog}: {catalog_ZCD.count} sources")

In [ ]:
if field == 'COSMOS':
    matches_ZCD = match.cross_match_catalogs_by_ra_dec(catalog_3DHST, catalog_ZCD)
    print(f"Number of Matches: {matches_ZCD.count}")

In [ ]:
if field == 'COSMOS':
    match.append_cross_matched_data(matches_ZCD, catalog_3DHST, catalog_ZCD)

In [ ]:
if field == 'COSMOS' and matches_ZCD.count > 0:
    value_filter       = (catalog_3DHST.idmap['id']  < 1e7) & (catalog_3DHST.idmap['ZCD'] >= 0)
    other_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['ZCD'] >= 0) & (catalog_3DHST.idmap['ZCD_dist']  > 0.0)
    multi_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['ZCD'] >= 0) & (catalog_3DHST.idmap['ZCD_dist'] == 0.0) & (catalog_3DHST.idmap['ZCD_num'] > 1)
    dup_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['ZCD'] >= 0) & (catalog_3DHST.idmap['ZCD_dist'] == 0.0) & (catalog_3DHST.idmap['ZCD_num'] == 1)
    non_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['ZCD'] >= 0) & (catalog_3DHST.idmap['ZCD_dist'] == 0.0) & (catalog_3DHST.idmap['ZCD_num'] == 0)

    num_matches = np.sum(value_filter)
    num_other_matches = np.sum(other_match_filter)
    num_multi_matches = np.sum(multi_match_filter)
    num_dup_matches = np.sum(dup_match_filter)
    num_non_matches = np.sum(non_match_filter)
    print(f"Number of {catalog_ZCD.name} Redshifts:")
    print(f"    Matches      : {num_matches} (diff={matches_ZCD.count - num_matches - num_other_matches})")
    print(f"  + Other matches: {num_other_matches}")
    print(f"  + Dup-matches  : {num_dup_matches}")
    print(f"  + Multi-matches: {num_multi_matches}")
    print(f"  + Non-matches  : {num_non_matches}")
    print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_ZCD.count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

    zlim_deep = [0, 6]
    zlim = [0, 3.5]

    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_ZCD'] > 0.0) & (catalog_3DHST.redshift['z_ZCD_flag'] < 8)
    fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_ZCD.catalog} vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_ZCD'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel(f"$z_{{peak}}$ [{catalog_3DHST.catalog}]")
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_ZCD.catalog}]")
    ax.set_xlim(zlim_deep)
    ax.set_ylim(zlim_deep)

    fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_ZCD.catalog}")
    if field == 'COSMOS' and matches_ZCB.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCB'] > 0.0) & (catalog_3DHST.redshift['z_ZCB_flag'] < 8) & (catalog_3DHST.redshift['z_ZCD'] > 0.0) & (catalog_3DHST.redshift['z_ZCD_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCB'][value_filter], catalog_3DHST.redshift['z_ZCD'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
    ax.set_xlabel('$z_{spec}$ [Other]')
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_ZCD.catalog}]")
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)
    ax.legend(loc='lower right')

    del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_ZCD' in locals():
    catalog.close(catalog_ZCD)
    del catalog_ZCD, catalog_params_ZCD

### Cross-Match VUDS (DR1, 2015-08-12)

In [ ]:
if field == 'COSMOS' or field == 'GOODS-S':
    catalog_params_VUDS = catalog.get_params('VUDS', field)
    catalog_VUDS = catalog.read(catalog_params_VUDS)
    print(f"{catalog_VUDS.catalog}: {catalog_VUDS.count} sources")

In [ ]:
if field == 'COSMOS' or field == 'GOODS-S':
    matches_VUDS = match.cross_match_catalogs_by_ra_dec(catalog_3DHST, catalog_VUDS)
    print(f"Number of Matches: {matches_VUDS.count}")

In [ ]:
if field == 'COSMOS' or field == 'GOODS-S':
    match.append_cross_matched_data(matches_VUDS, catalog_3DHST, catalog_VUDS)

In [ ]:
if (field == 'COSMOS' or field == 'GOODS-S') and matches_VUDS.count > 0:
    value_filter       = (catalog_3DHST.idmap['id']  < 1e7) & (catalog_3DHST.idmap['VUDS'] >= 0)
    other_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['VUDS'] >= 0) & (catalog_3DHST.idmap['VUDS_dist']  > 0.0)
    multi_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['VUDS'] >= 0) & (catalog_3DHST.idmap['VUDS_dist'] == 0.0) & (catalog_3DHST.idmap['VUDS_num'] > 1)
    dup_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['VUDS'] >= 0) & (catalog_3DHST.idmap['VUDS_dist'] == 0.0) & (catalog_3DHST.idmap['VUDS_num'] == 1)
    non_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['VUDS'] >= 0) & (catalog_3DHST.idmap['VUDS_dist'] == 0.0) & (catalog_3DHST.idmap['VUDS_num'] == 0)

    num_matches = np.sum(value_filter)
    num_other_matches = np.sum(other_match_filter)
    num_multi_matches = np.sum(multi_match_filter)
    num_dup_matches = np.sum(dup_match_filter)
    num_non_matches = np.sum(non_match_filter)
    print(f"Number of {catalog_VUDS.name} Redshifts:")
    print(f"    Matches      : {num_matches} (diff={matches_VUDS.count - num_matches - num_other_matches})")
    print(f"  + Other matches: {num_other_matches}")
    print(f"  + Dup-matches  : {num_dup_matches}")
    print(f"  + Multi-matches: {num_multi_matches}")
    print(f"  + Non-matches  : {num_non_matches}")
    print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_VUDS.count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

    zlim_deep = [0, 6]
    zlim = [0, 6]

    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_VUDS'] > 0.0) & (catalog_3DHST.redshift['z_VUDS_flag'] < 8)
    fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_VUDS.catalog} vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_VUDS'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel(f"$z_{{peak}}$ [{catalog_3DHST.catalog}]")
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_VUDS.catalog}]")
    ax.set_xlim(zlim_deep)
    ax.set_ylim(zlim_deep)

    fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_VUDS.catalog}")
    if field == 'COSMOS' and matches_ZCB.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCB'] > 0.0) & (catalog_3DHST.redshift['z_ZCB_flag'] < 8) & (catalog_3DHST.redshift['z_VUDS'] > 0.0) & (catalog_3DHST.redshift['z_VUDS_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCB'][value_filter], catalog_3DHST.redshift['z_VUDS'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
    if field == 'COSMOS' and matches_ZCD.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCD'] > 0.0) & (catalog_3DHST.redshift['z_ZCD_flag'] < 8) & (catalog_3DHST.redshift['z_VUDS'] > 0.0) & (catalog_3DHST.redshift['z_VUDS_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCD'][value_filter], catalog_3DHST.redshift['z_VUDS'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
    ax.set_xlabel('$z_{spec}$ [Other]')
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_VUDS.catalog}]")
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)
    ax.legend(loc='lower right')

    del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_VUDS' in locals():
    catalog.close(catalog_VUDS)
    del catalog_VUDS, catalog_params_VUDS

### Cross-Match Casey DSFG (2017-05-10)

In [ ]:
if field == 'COSMOS' or field == 'GOODS-N' or field == 'UDS':
    catalog_params_Casey = catalog.get_params('Casey', field)
    catalog_Casey = catalog.read(catalog_params_Casey)
    print(f"{catalog_Casey.catalog}: {catalog_Casey.count} sources")

In [ ]:
if field == 'COSMOS' or field == 'GOODS-N' or field == 'UDS':
    matches_Casey = match.cross_match_catalogs_by_ra_dec(catalog_3DHST, catalog_Casey)
    print(f"Number of Matches: {matches_Casey.count}")

In [ ]:
if field == 'COSMOS' or field == 'GOODS-N' or field == 'UDS':
    match.append_cross_matched_data(matches_Casey, catalog_3DHST, catalog_Casey)

In [ ]:
if (field == 'COSMOS' or field == 'GOODS-N' or field == 'UDS') and matches_Casey.count > 0:
    value_filter       = (catalog_3DHST.idmap['id']  < 1e7) & (catalog_3DHST.idmap['Casey'] >= 0)
    other_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['Casey'] >= 0) & (catalog_3DHST.idmap['Casey_dist'] > 0.0)
    multi_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['Casey'] >= 0) & (catalog_3DHST.idmap['Casey_dist'] == 0.0) & (catalog_3DHST.idmap['Casey_num'] > 1)
    dup_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['Casey'] >= 0) & (catalog_3DHST.idmap['Casey_dist'] == 0.0) & (catalog_3DHST.idmap['Casey_num'] == 1)
    non_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['Casey'] >= 0) & (catalog_3DHST.idmap['Casey_dist'] == 0.0) & (catalog_3DHST.idmap['Casey_num'] == 0)

    num_matches = np.sum(value_filter)
    num_other_matches = np.sum(other_match_filter)
    num_multi_matches = np.sum(multi_match_filter)
    num_dup_matches = np.sum(dup_match_filter)
    num_non_matches = np.sum(non_match_filter)
    print(f"Number of {catalog_Casey.name} Redshifts:")
    print(f"    Matches      : {num_matches} (diff={matches_Casey.count - num_matches - num_other_matches})")
    print(f"  + Other matches: {num_other_matches}")
    print(f"  + Dup-matches  : {num_dup_matches}")
    print(f"  + Multi-matches: {num_multi_matches}")
    print(f"  + Non-matches  : {num_non_matches}")
    print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_Casey.count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

    zlim_deep = [0, 3.5]
    zlim = [0, 1.5]

    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_Casey'] > 0.0) & (catalog_3DHST.redshift['z_Casey_flag'] < 8)
    fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_Casey.catalog} vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_Casey'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel(f"$z_{{peak}}$ [{catalog_3DHST.catalog}]")
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_Casey.catalog}]")
    ax.set_xlim(zlim_deep)
    ax.set_ylim(zlim_deep)

    fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_Casey.catalog}")
    if field == 'COSMOS' and matches_ZCB.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCB'] > 0.0) & (catalog_3DHST.redshift['z_ZCB_flag'] < 8) & (catalog_3DHST.redshift['z_Casey'] > 0.0) & (catalog_3DHST.redshift['z_Casey_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCB'][value_filter], catalog_3DHST.redshift['z_Casey'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
    if field == 'COSMOS' and matches_ZCD.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCD'] > 0.0) & (catalog_3DHST.redshift['z_ZCD_flag'] < 8) & (catalog_3DHST.redshift['z_Casey'] > 0.0) & (catalog_3DHST.redshift['z_Casey_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCD'][value_filter], catalog_3DHST.redshift['z_Casey'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
    if (field == 'COSMOS' or field == 'GOODS-S') and matches_VUDS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_VUDS'] > 0.0) & (catalog_3DHST.redshift['z_VUDS_flag'] < 8) & (catalog_3DHST.redshift['z_Casey'] > 0.0) & (catalog_3DHST.redshift['z_Casey_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_VUDS'][value_filter], catalog_3DHST.redshift['z_Casey'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
    ax.set_xlabel('$z_{spec}$ [Other]')
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_Casey.catalog}]")
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)
    ax.legend(loc='lower right')

    del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_Casey' in locals():
    catalog.close(catalog_Casey)
    del catalog_Casey, catalog_params_Casey

### Cross-Match DEIMOS 10K (2017-12-27)

In [ ]:
if field == 'COSMOS':
    catalog_params_DEIMOS = catalog.get_params('DEIMOS')
    catalog_DEIMOS = catalog.read(catalog_params_DEIMOS)
    print(f"{catalog_DEIMOS.catalog}: {catalog_DEIMOS.count} sources")

In [ ]:
if field == 'COSMOS':
    matches_DEIMOS = match.cross_match_catalogs_by_ra_dec(catalog_3DHST, catalog_DEIMOS)
    print(f"Number of Matches: {matches_DEIMOS.count}")

In [ ]:
if field == 'COSMOS':
    match.append_cross_matched_data(matches_DEIMOS, catalog_3DHST, catalog_DEIMOS)

In [ ]:
if field == 'COSMOS' and matches_DEIMOS.count > 0:
    value_filter       = (catalog_3DHST.idmap['id']  < 1e7) & (catalog_3DHST.idmap['DEIMOS'] >= 0)
    other_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['DEIMOS'] >= 0) & (catalog_3DHST.idmap['DEIMOS_dist'] > 0.0)
    multi_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['DEIMOS'] >= 0) & (catalog_3DHST.idmap['DEIMOS_dist'] == 0.0) & (catalog_3DHST.idmap['DEIMOS_num'] > 1)
    dup_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['DEIMOS'] >= 0) & (catalog_3DHST.idmap['DEIMOS_dist'] == 0.0) & (catalog_3DHST.idmap['DEIMOS_num'] == 1)
    non_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['DEIMOS'] >= 0) & (catalog_3DHST.idmap['DEIMOS_dist'] == 0.0) & (catalog_3DHST.idmap['DEIMOS_num'] == 0)

    num_matches = np.sum(value_filter)
    num_other_matches = np.sum(other_match_filter)
    num_multi_matches = np.sum(multi_match_filter)
    num_dup_matches = np.sum(dup_match_filter)
    num_non_matches = np.sum(non_match_filter)
    print(f"Number of {catalog_DEIMOS.name} Redshifts:")
    print(f"    Matches      : {num_matches} (diff={matches_DEIMOS.count - num_matches - num_other_matches})")
    print(f"  + Other matches: {num_other_matches}")
    print(f"  + Dup-matches  : {num_dup_matches}")
    print(f"  + Multi-matches: {num_multi_matches}")
    print(f"  + Non-matches  : {num_non_matches}")
    print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_DEIMOS.count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

    zlim_deep = [0, 6]
    zlim = [0, 6]

    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS_flag'] < 8)
    fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_DEIMOS.catalog} vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_DEIMOS'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel(f"$z_{{peak}}$ [{catalog_3DHST.catalog}]")
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_DEIMOS.catalog}]")
    ax.set_xlim(zlim_deep)
    ax.set_ylim(zlim_deep)

    fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_DEIMOS.catalog}")
    if field == 'COSMOS' and matches_ZCB.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCB'] > 0.0) & (catalog_3DHST.redshift['z_ZCB_flag'] < 8) & (catalog_3DHST.redshift['z_DEIMOS'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCB'][value_filter], catalog_3DHST.redshift['z_DEIMOS'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
    if field == 'COSMOS' and matches_ZCD.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCD'] > 0.0) & (catalog_3DHST.redshift['z_ZCD_flag'] < 8) & (catalog_3DHST.redshift['z_DEIMOS'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCD'][value_filter], catalog_3DHST.redshift['z_DEIMOS'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
    if (field == 'COSMOS' or field == 'GOODS-S') and matches_VUDS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_VUDS'] > 0.0) & (catalog_3DHST.redshift['z_VUDS_flag'] < 8) & (catalog_3DHST.redshift['z_DEIMOS'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_VUDS'][value_filter], catalog_3DHST.redshift['z_DEIMOS'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'COSMOS' or field == 'GOODS-N' or field == 'UDS') and matches_Casey.count > 0:
        value_filter = (catalog_3DHST.redshift['z_Casey'] > 0.0) & (catalog_3DHST.redshift['z_Casey_flag'] < 8) & (catalog_3DHST.redshift['z_DEIMOS'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_Casey'][value_filter], catalog_3DHST.redshift['z_DEIMOS'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
    ax.set_xlabel('$z_{spec}$ [Other]')
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_DEIMOS.catalog}]")
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)
    ax.legend(loc='lower right')

    del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_DEIMOS' in locals():
    catalog.close(catalog_DEIMOS)
    del catalog_DEIMOS, catalog_params_DEIMOS

### Cross-Match MOSDEF (Final, 2018-03-11)

In [ ]:
if field == 'AEGIS' or field == 'COSMOS' or field == 'GOODS-N' or field == 'GOODS-S' or field == 'UDS':
    catalog_params_MOSDEF = catalog.get_params('MOSDEF', field)
    catalog_MOSDEF = catalog.read(catalog_params_MOSDEF)
    print(f"{catalog_MOSDEF.catalog}: {catalog_MOSDEF.count} sources")

In [ ]:
if field == 'AEGIS' or field == 'COSMOS' or field == 'GOODS-N' or field == 'GOODS-S' or field == 'UDS':
    matches_MOSDEF = match.cross_match_catalogs_by_id(catalog_3DHST, catalog_MOSDEF, id_field2='ID_V4')
    print(f"Number of Matches: {matches_MOSDEF.count}")

In [ ]:
if field == 'AEGIS' or field == 'COSMOS' or field == 'GOODS-N' or field == 'GOODS-S' or field == 'UDS':
    match.append_cross_matched_data(matches_MOSDEF, catalog_3DHST, catalog_MOSDEF)

In [ ]:
if (field == 'AEGIS' or field == 'COSMOS' or field == 'GOODS-N' or field == 'GOODS-S' or field == 'UDS') and matches_MOSDEF.count > 0:
    value_filter       = (catalog_3DHST.idmap['id']  < 1e7) & (catalog_3DHST.idmap['MOSDEF'] >= 0)
    other_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['MOSDEF'] >= 0) & (catalog_3DHST.idmap['MOSDEF_dist'] > 0.0)
    multi_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['MOSDEF'] >= 0) & (catalog_3DHST.idmap['MOSDEF_dist'] == 0.0) & (catalog_3DHST.idmap['MOSDEF_num'] > 1)
    dup_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['MOSDEF'] >= 0) & (catalog_3DHST.idmap['MOSDEF_dist'] == 0.0) & (catalog_3DHST.idmap['MOSDEF_num'] == 1)
    non_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['MOSDEF'] >= 0) & (catalog_3DHST.idmap['MOSDEF_dist'] == 0.0) & (catalog_3DHST.idmap['MOSDEF_num'] == 0)

    num_matches = np.sum(value_filter)
    num_other_matches = np.sum(other_match_filter)
    num_multi_matches = np.sum(multi_match_filter)
    num_dup_matches = np.sum(dup_match_filter)
    num_non_matches = np.sum(non_match_filter)
    print(f"Number of {catalog_MOSDEF.name} Redshifts:")
    print(f"    Matches      : {num_matches} (diff={matches_MOSDEF.count - num_matches - num_other_matches})")
    print(f"  + Other matches: {num_other_matches}")
    print(f"  + Dup-matches  : {num_dup_matches}")
    print(f"  + Multi-matches: {num_multi_matches}")
    print(f"  + Non-matches  : {num_non_matches}")
    print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_MOSDEF.count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

    zlim_deep = [0, 4]
    zlim = [0, 4]

    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF_flag'] < 8)
    fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_MOSDEF.catalog} vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_MOSDEF'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel(f"$z_{{peak}}$ [{catalog_3DHST.catalog}]")
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_MOSDEF.catalog}]")
    ax.set_xlim(zlim_deep)
    ax.set_ylim(zlim_deep)

    fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_MOSDEF.catalog}")
    if field == 'COSMOS' and matches_ZCB.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCB'] > 0.0) & (catalog_3DHST.redshift['z_ZCB_flag'] < 8) & (catalog_3DHST.redshift['z_MOSDEF'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCB'][value_filter], catalog_3DHST.redshift['z_MOSDEF'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
    if field == 'COSMOS' and matches_ZCD.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCD'] > 0.0) & (catalog_3DHST.redshift['z_ZCD_flag'] < 8) & (catalog_3DHST.redshift['z_MOSDEF'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCD'][value_filter], catalog_3DHST.redshift['z_MOSDEF'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
    if (field == 'COSMOS' or field == 'GOODS-S') and matches_VUDS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_VUDS'] > 0.0) & (catalog_3DHST.redshift['z_VUDS_flag'] < 8) & (catalog_3DHST.redshift['z_MOSDEF'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_VUDS'][value_filter], catalog_3DHST.redshift['z_MOSDEF'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'COSMOS' or field == 'GOODS-N' or field == 'UDS') and matches_Casey.count > 0:
        value_filter = (catalog_3DHST.redshift['z_Casey'] > 0.0) & (catalog_3DHST.redshift['z_Casey_flag'] < 8) & (catalog_3DHST.redshift['z_MOSDEF'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_Casey'][value_filter], catalog_3DHST.redshift['z_MOSDEF'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
    if field == 'COSMOS' and matches_DEIMOS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_DEIMOS'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS_flag'] < 8) & (catalog_3DHST.redshift['z_MOSDEF'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_DEIMOS'][value_filter], catalog_3DHST.redshift['z_MOSDEF'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour5)
    ax.set_xlabel('$z_{spec}$ [Other]')
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_MOSDEF.catalog}]")
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)
    ax.legend(loc='lower right')

    del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_MOSDEF' in locals():
    catalog.close(catalog_MOSDEF)
    del catalog_MOSDEF, catalog_params_MOSDEF

### Cross-Match FMOS (v2, 2019-01-25)

In [ ]:
if field == 'COSMOS':
    catalog_params_FMOS = catalog.get_params('FMOS')
    catalog_FMOS = catalog.read(catalog_params_FMOS)
    print(f"{catalog_FMOS.catalog}: {catalog_FMOS.count} sources")

In [ ]:
if field == 'COSMOS':
    matches_FMOS = match.cross_match_catalogs_by_ra_dec(catalog_3DHST, catalog_FMOS)
    print(f"Number of Matches: {matches_FMOS.count}")

In [ ]:
if field == 'COSMOS':
    match.append_cross_matched_data(matches_FMOS, catalog_3DHST, catalog_FMOS)

In [ ]:
if field == 'COSMOS' and matches_FMOS.count > 0:
    value_filter       = (catalog_3DHST.idmap['id']  < 1e7) & (catalog_3DHST.idmap['FMOS'] >= 0)
    other_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['FMOS'] >= 0) & (catalog_3DHST.idmap['FMOS_dist'] > 0.0)
    multi_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['FMOS'] >= 0) & (catalog_3DHST.idmap['FMOS_dist'] == 0.0) & (catalog_3DHST.idmap['FMOS_num'] > 1)
    dup_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['FMOS'] >= 0) & (catalog_3DHST.idmap['FMOS_dist'] == 0.0) & (catalog_3DHST.idmap['FMOS_num'] == 1)
    non_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['FMOS'] >= 0) & (catalog_3DHST.idmap['FMOS_dist'] == 0.0) & (catalog_3DHST.idmap['FMOS_num'] == 0)

    num_matches = np.sum(value_filter)
    num_other_matches = np.sum(other_match_filter)
    num_multi_matches = np.sum(multi_match_filter)
    num_dup_matches = np.sum(dup_match_filter)
    num_non_matches = np.sum(non_match_filter)
    print(f"Number of {catalog_FMOS.name} Redshifts:")
    print(f"    Matches      : {num_matches} (diff={matches_FMOS.count - num_matches - num_other_matches})")
    print(f"  + Other matches: {num_other_matches}")
    print(f"  + Dup-matches  : {num_dup_matches}")
    print(f"  + Multi-matches: {num_multi_matches}")
    print(f"  + Non-matches  : {num_non_matches}")
    print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_FMOS.count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

    zlim_deep = [0, 3]
    zlim = [0, 4]

    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_FMOS'] > 0.0) & (catalog_3DHST.redshift['z_FMOS_flag'] < 8)
    fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_FMOS.catalog} vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_FMOS'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel(f"$z_{{peak}}$ [{catalog_3DHST.catalog}]")
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_FMOS.catalog}]")
    ax.set_xlim(zlim_deep)
    ax.set_ylim(zlim_deep)

    fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_FMOS.catalog}")
    if field == 'COSMOS' and matches_ZCB.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCB'] > 0.0) & (catalog_3DHST.redshift['z_ZCB_flag'] < 8) & (catalog_3DHST.redshift['z_FMOS'] > 0.0) & (catalog_3DHST.redshift['z_FMOS_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCB'][value_filter], catalog_3DHST.redshift['z_FMOS'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
    if field == 'COSMOS' and matches_ZCD.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCD'] > 0.0) & (catalog_3DHST.redshift['z_ZCD_flag'] < 8) & (catalog_3DHST.redshift['z_FMOS'] > 0.0) & (catalog_3DHST.redshift['z_FMOS_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCD'][value_filter], catalog_3DHST.redshift['z_FMOS'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
    if (field == 'COSMOS' or field == 'GOODS-S') and matches_VUDS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_VUDS'] > 0.0) & (catalog_3DHST.redshift['z_VUDS_flag'] < 8) & (catalog_3DHST.redshift['z_FMOS'] > 0.0) & (catalog_3DHST.redshift['z_FMOS_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_VUDS'][value_filter], catalog_3DHST.redshift['z_FMOS'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'COSMOS' or field == 'GOODS-N' or field == 'UDS') and matches_Casey.count > 0:
        value_filter = (catalog_3DHST.redshift['z_Casey'] > 0.0) & (catalog_3DHST.redshift['z_Casey_flag'] < 8) & (catalog_3DHST.redshift['z_FMOS'] > 0.0) & (catalog_3DHST.redshift['z_FMOS_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_Casey'][value_filter], catalog_3DHST.redshift['z_FMOS'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
    if field == 'COSMOS' and matches_DEIMOS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_DEIMOS'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS_flag'] < 8) & (catalog_3DHST.redshift['z_FMOS'] > 0.0) & (catalog_3DHST.redshift['z_FMOS_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_DEIMOS'][value_filter], catalog_3DHST.redshift['z_FMOS'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'AEGIS' or field == 'COSMOS' or field == 'GOODS-N' or field == 'GOODS-S' or field == 'UDS') and matches_MOSDEF.count > 0:
        value_filter = (catalog_3DHST.redshift['z_MOSDEF'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF_flag'] < 8) & (catalog_3DHST.redshift['z_FMOS'] > 0.0) & (catalog_3DHST.redshift['z_FMOS_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_MOSDEF'][value_filter], catalog_3DHST.redshift['z_FMOS'][value_filter], label=f"MOSDEF ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
    ax.set_xlabel('$z_{spec}$ [Other]')
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_FMOS.catalog}]")
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)
    ax.legend(loc='lower right')

    del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_FMOS' in locals():
    catalog.close(catalog_FMOS)
    del catalog_FMOS, catalog_params_FMOS

### Cross-Match KMOS3D (2019-07-12)

In [ ]:
if field == 'COSMOS' or field == 'GOODS-S' or field == 'UDS':
    catalog_params_KMOS3D = catalog.get_params('KMOS3D', field)
    catalog_KMOS3D = catalog.read(catalog_params_KMOS3D)
    print(f"{catalog_KMOS3D.catalog}: {catalog_KMOS3D.count} sources")

In [ ]:
if field == 'COSMOS' or field == 'GOODS-S' or field == 'UDS':
    matches_KMOS3D = match.cross_match_catalogs_by_id(catalog_3DHST, catalog_KMOS3D)
    print(f"Number of Matches: {matches_KMOS3D.count}")

In [ ]:
if field == 'COSMOS' or field == 'GOODS-S' or field == 'UDS':
    match.append_cross_matched_data(matches_KMOS3D, catalog_3DHST, catalog_KMOS3D)

In [ ]:
if (field == 'COSMOS' or field == 'GOODS-S' or field == 'UDS') and matches_KMOS3D.count > 0:
    value_filter       = (catalog_3DHST.idmap['id']  < 1e7) & (catalog_3DHST.idmap['KMOS3D'] >= 0)
    other_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['KMOS3D'] >= 0) & (catalog_3DHST.idmap['KMOS3D_dist']  > 0.0)
    multi_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['KMOS3D'] >= 0) & (catalog_3DHST.idmap['KMOS3D_dist'] == 0.0) & (catalog_3DHST.idmap['KMOS3D_num'] > 1)
    dup_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['KMOS3D'] >= 0) & (catalog_3DHST.idmap['KMOS3D_dist'] == 0.0) & (catalog_3DHST.idmap['KMOS3D_num'] == 1)
    non_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['KMOS3D'] >= 0) & (catalog_3DHST.idmap['KMOS3D_dist'] == 0.0) & (catalog_3DHST.idmap['KMOS3D_num'] == 0)

    num_matches = np.sum(value_filter)
    num_other_matches = np.sum(other_match_filter)
    num_multi_matches = np.sum(multi_match_filter)
    num_dup_matches = np.sum(dup_match_filter)
    num_non_matches = np.sum(non_match_filter)
    print(f"Number of {catalog_KMOS3D.name} Redshifts:")
    print(f"    Matches      : {num_matches} (diff={matches_KMOS3D.count - num_matches - num_other_matches})")
    print(f"  + Other matches: {num_other_matches}")
    print(f"  + Dup-matches  : {num_dup_matches}")
    print(f"  + Multi-matches: {num_multi_matches}")
    print(f"  + Non-matches  : {num_non_matches}")
    print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_KMOS3D.count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

    zlim_deep = [0, 4]
    zlim = [0, 3.5]

    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D_flag'] < 8)
    fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_KMOS3D.catalog} vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_KMOS3D'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel(f"$z_{{peak}}$ [{catalog_3DHST.catalog}]")
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_KMOS3D.catalog}]")
    ax.set_xlim(zlim_deep)
    ax.set_ylim(zlim_deep)

    fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_KMOS3D.catalog}")
    if field == 'COSMOS' and matches_ZCB.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCB'] > 0.0) & (catalog_3DHST.redshift['z_ZCB_flag'] < 8) & (catalog_3DHST.redshift['z_KMOS3D'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCB'][value_filter], catalog_3DHST.redshift['z_KMOS3D'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
    if field == 'COSMOS' and matches_ZCD.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCD'] > 0.0) & (catalog_3DHST.redshift['z_ZCD_flag'] < 8) & (catalog_3DHST.redshift['z_KMOS3D'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCD'][value_filter], catalog_3DHST.redshift['z_KMOS3D'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
    if (field == 'COSMOS' or field == 'GOODS-S') and matches_VUDS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_VUDS'] > 0.0) & (catalog_3DHST.redshift['z_VUDS_flag'] < 8) & (catalog_3DHST.redshift['z_KMOS3D'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_VUDS'][value_filter], catalog_3DHST.redshift['z_KMOS3D'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'COSMOS' or field == 'GOODS-N' or field == 'UDS') and matches_Casey.count > 0:
        value_filter = (catalog_3DHST.redshift['z_Casey'] > 0.0) & (catalog_3DHST.redshift['z_Casey_flag'] < 8) & (catalog_3DHST.redshift['z_KMOS3D'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_Casey'][value_filter], catalog_3DHST.redshift['z_KMOS3D'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
    if field == 'COSMOS' and matches_DEIMOS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_DEIMOS'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS_flag'] < 8) & (catalog_3DHST.redshift['z_KMOS3D'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_DEIMOS'][value_filter], catalog_3DHST.redshift['z_KMOS3D'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'AEGIS' or field == 'COSMOS' or field == 'GOODS-N' or field == 'GOODS-S' or field == 'UDS') and matches_MOSDEF.count > 0:
        value_filter = (catalog_3DHST.redshift['z_MOSDEF'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF_flag'] < 8) & (catalog_3DHST.redshift['z_KMOS3D'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_MOSDEF'][value_filter], catalog_3DHST.redshift['z_KMOS3D'][value_filter], label=f"MOSDEF ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
    if field == 'COSMOS' and matches_FMOS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_FMOS'] > 0.0) & (catalog_3DHST.redshift['z_FMOS_flag'] < 8) & (catalog_3DHST.redshift['z_KMOS3D'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_FMOS'][value_filter], catalog_3DHST.redshift['z_KMOS3D'][value_filter], label=f"FMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3)
    ax.set_xlabel('$z_{spec}$ [Other]')
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_KMOS3D.catalog}]")
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)
    ax.legend(loc='lower right')

    del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_KMOS3D' in locals():
    catalog.close(catalog_KMOS3D)
    del catalog_KMOS3D, catalog_params_KMOS3D

### Cross-Match C3R2 (DR1+DR2, 2019-04-11 + DR3, 2021-06-18)

In [ ]:
if field == 'COSMOS':
    catalog_params_C3R2 = catalog.get_params('C3R2', field)
    catalog_C3R2 = catalog.read(catalog_params_C3R2)
    print(f"{catalog_C3R2.catalog}: {catalog_C3R2.count} sources")

In [ ]:
if field == 'COSMOS':
    matches_C3R2 = match.cross_match_catalogs_by_ra_dec(catalog_3DHST, catalog_C3R2)
    print(f"Number of Matches: {matches_C3R2.count}")

In [ ]:
if field == 'COSMOS':
    match.append_cross_matched_data(matches_C3R2, catalog_3DHST, catalog_C3R2)

In [ ]:
if field == 'COSMOS' and matches_C3R2.count > 0:
    value_filter       = (catalog_3DHST.idmap['id']  < 1e7) & (catalog_3DHST.idmap['C3R2'] >= 0)
    other_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['C3R2'] >= 0) & (catalog_3DHST.idmap['C3R2_dist'] > 0.0)
    multi_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['C3R2'] >= 0) & (catalog_3DHST.idmap['C3R2_dist'] == 0.0) & (catalog_3DHST.idmap['C3R2_num'] > 1)
    dup_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['C3R2'] >= 0) & (catalog_3DHST.idmap['C3R2_dist'] == 0.0) & (catalog_3DHST.idmap['C3R2_num'] == 1)
    non_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['C3R2'] >= 0) & (catalog_3DHST.idmap['C3R2_dist'] == 0.0) & (catalog_3DHST.idmap['C3R2_num'] == 0)

    num_matches = np.sum(value_filter)
    num_other_matches = np.sum(other_match_filter)
    num_multi_matches = np.sum(multi_match_filter)
    num_dup_matches = np.sum(dup_match_filter)
    num_non_matches = np.sum(non_match_filter)
    print(f"Number of {catalog_C3R2.name} Redshifts:")
    print(f"    Matches      : {num_matches} (diff={matches_C3R2.count - num_matches - num_other_matches})")
    print(f"  + Other matches: {num_other_matches}")
    print(f"  + Dup-matches  : {num_dup_matches}")
    print(f"  + Multi-matches: {num_multi_matches}")
    print(f"  + Non-matches  : {num_non_matches}")
    print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_C3R2.count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

    zlim_deep = [0, 4]
    zlim = [0, 3.5]

    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_C3R2'] > 0.0) & (catalog_3DHST.redshift['z_C3R2_flag'] < 8)
    fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_C3R2.catalog} vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_C3R2'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel(f"$z_{{peak}}$ [{catalog_3DHST.catalog}]")
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_C3R2.catalog}]")
    ax.set_xlim(zlim_deep)
    ax.set_ylim(zlim_deep)

    fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_C3R2.catalog}")
    if field == 'COSMOS' and matches_ZCB.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCB'] > 0.0) & (catalog_3DHST.redshift['z_ZCB_flag'] < 8) & (catalog_3DHST.redshift['z_C3R2'] > 0.0) & (catalog_3DHST.redshift['z_C3R2_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCB'][value_filter], catalog_3DHST.redshift['z_C3R2'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
    if field == 'COSMOS' and matches_ZCD.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCD'] > 0.0) & (catalog_3DHST.redshift['z_ZCD_flag'] < 8) & (catalog_3DHST.redshift['z_C3R2'] > 0.0) & (catalog_3DHST.redshift['z_C3R2_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCD'][value_filter], catalog_3DHST.redshift['z_C3R2'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
    if (field == 'COSMOS' or field == 'GOODS-S') and matches_VUDS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_VUDS'] > 0.0) & (catalog_3DHST.redshift['z_VUDS_flag'] < 8) & (catalog_3DHST.redshift['z_C3R2'] > 0.0) & (catalog_3DHST.redshift['z_C3R2_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_VUDS'][value_filter], catalog_3DHST.redshift['z_C3R2'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'COSMOS' or field == 'GOODS-N' or field == 'UDS') and matches_Casey.count > 0:
        value_filter = (catalog_3DHST.redshift['z_Casey'] > 0.0) & (catalog_3DHST.redshift['z_Casey_flag'] < 8) & (catalog_3DHST.redshift['z_C3R2'] > 0.0) & (catalog_3DHST.redshift['z_C3R2_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_Casey'][value_filter], catalog_3DHST.redshift['z_C3R2'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
    if field == 'COSMOS' and matches_DEIMOS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_DEIMOS'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS_flag'] < 8) & (catalog_3DHST.redshift['z_C3R2'] > 0.0) & (catalog_3DHST.redshift['z_C3R2_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_DEIMOS'][value_filter], catalog_3DHST.redshift['z_C3R2'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'AEGIS' or field == 'COSMOS' or field == 'GOODS-N' or field == 'GOODS-S' or field == 'UDS') and matches_MOSDEF.count > 0:
        value_filter = (catalog_3DHST.redshift['z_MOSDEF'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF_flag'] < 8) & (catalog_3DHST.redshift['z_C3R2'] > 0.0) & (catalog_3DHST.redshift['z_C3R2_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_MOSDEF'][value_filter], catalog_3DHST.redshift['z_C3R2'][value_filter], label=f"MOSDEF ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
    if field == 'COSMOS' and matches_FMOS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_FMOS'] > 0.0) & (catalog_3DHST.redshift['z_FMOS_flag'] < 8) & (catalog_3DHST.redshift['z_C3R2'] > 0.0) & (catalog_3DHST.redshift['z_C3R2_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_FMOS'][value_filter], catalog_3DHST.redshift['z_C3R2'][value_filter], label=f"FMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3)
    if (field == 'COSMOS' or field == 'GOODS-S' or field == 'UDS') and matches_KMOS3D.count > 0:
        value_filter = (catalog_3DHST.redshift['z_KMOS3D'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D_flag'] < 8) & (catalog_3DHST.redshift['z_C3R2'] > 0.0) & (catalog_3DHST.redshift['z_C3R2_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_KMOS3D'][value_filter], catalog_3DHST.redshift['z_C3R2'][value_filter], label=f"KMOS3D ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3)
    ax.set_xlabel('$z_{spec}$ [Other]')
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_C3R2.catalog}]")
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)
    ax.legend(loc='lower right')

    del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_C3R2' in locals():
    catalog.close(catalog_C3R2)
    del catalog_C3R2, catalog_params_C3R2

### Cross-Match LEGA-C (DR3, 2021-08-02)

In [ ]:
if field == 'COSMOS':
    catalog_params_LEGAC = catalog.get_params('LEGAC')
    catalog_LEGAC = catalog.read(catalog_params_LEGAC)
    print(f"{catalog_LEGAC.catalog}: {catalog_LEGAC.count} sources")

In [ ]:
if field == 'COSMOS':
    matches_LEGAC = match.cross_match_catalogs_by_ra_dec(catalog_3DHST, catalog_LEGAC)
    print(f"Number of Matches: {matches_LEGAC.count}")

In [ ]:
if field == 'COSMOS':
    match.append_cross_matched_data(matches_LEGAC, catalog_3DHST, catalog_LEGAC)

In [ ]:
if field == 'COSMOS' and matches_LEGAC.count > 0:
    value_filter       = (catalog_3DHST.idmap['id']  < 1e7) & (catalog_3DHST.idmap['LEGAC'] >= 0)
    other_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['LEGAC'] >= 0) & (catalog_3DHST.idmap['LEGAC_dist'] > 0.0)
    multi_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['LEGAC'] >= 0) & (catalog_3DHST.idmap['LEGAC_dist'] == 0.0) & (catalog_3DHST.idmap['LEGAC_num'] > 1)
    dup_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['LEGAC'] >= 0) & (catalog_3DHST.idmap['LEGAC_dist'] == 0.0) & (catalog_3DHST.idmap['LEGAC_num'] == 1)
    non_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['LEGAC'] >= 0) & (catalog_3DHST.idmap['LEGAC_dist'] == 0.0) & (catalog_3DHST.idmap['LEGAC_num'] == 0)

    num_matches = np.sum(value_filter)
    num_other_matches = np.sum(other_match_filter)
    num_multi_matches = np.sum(multi_match_filter)
    num_dup_matches = np.sum(dup_match_filter)
    num_non_matches = np.sum(non_match_filter)
    print(f"Number of {catalog_LEGAC.name} Redshifts:")
    print(f"    Matches      : {num_matches} (diff={matches_LEGAC.count - num_matches - num_other_matches})")
    print(f"  + Other matches: {num_other_matches}")
    print(f"  + Dup-matches  : {num_dup_matches}")
    print(f"  + Multi-matches: {num_multi_matches}")
    print(f"  + Non-matches  : {num_non_matches}")
    print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_LEGAC.count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

    zlim_deep = [0, 3]
    zlim = [0, 2]

    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC_flag'] < 8)
    fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_LEGAC.catalog} vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_LEGAC'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel(f"$z_{{peak}}$ [{catalog_3DHST.catalog}]")
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_LEGAC.catalog}]")
    ax.set_xlim(zlim_deep)
    ax.set_ylim(zlim_deep)

    fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_LEGAC.catalog}")
    if field == 'COSMOS' and matches_ZCB.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCB'] > 0.0) & (catalog_3DHST.redshift['z_ZCB_flag'] < 8) & (catalog_3DHST.redshift['z_LEGAC'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCB'][value_filter], catalog_3DHST.redshift['z_LEGAC'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
    if field == 'COSMOS' and matches_ZCD.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCD'] > 0.0) & (catalog_3DHST.redshift['z_ZCD_flag'] < 8) & (catalog_3DHST.redshift['z_LEGAC'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCD'][value_filter], catalog_3DHST.redshift['z_LEGAC'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
    if (field == 'COSMOS' or field == 'GOODS-S') and matches_VUDS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_VUDS'] > 0.0) & (catalog_3DHST.redshift['z_VUDS_flag'] < 8) & (catalog_3DHST.redshift['z_LEGAC'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_VUDS'][value_filter], catalog_3DHST.redshift['z_LEGAC'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'COSMOS' or field == 'GOODS-N' or field == 'UDS') and matches_Casey.count > 0:
        value_filter = (catalog_3DHST.redshift['z_Casey'] > 0.0) & (catalog_3DHST.redshift['z_Casey_flag'] < 8) & (catalog_3DHST.redshift['z_LEGAC'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_Casey'][value_filter], catalog_3DHST.redshift['z_LEGAC'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
    if field == 'COSMOS' and matches_DEIMOS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_DEIMOS'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS_flag'] < 8) & (catalog_3DHST.redshift['z_LEGAC'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_DEIMOS'][value_filter], catalog_3DHST.redshift['z_LEGAC'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'AEGIS' or field == 'COSMOS' or field == 'GOODS-N' or field == 'GOODS-S' or field == 'UDS') and matches_MOSDEF.count > 0:
        value_filter = (catalog_3DHST.redshift['z_MOSDEF'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF_flag'] < 8) & (catalog_3DHST.redshift['z_LEGAC'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_MOSDEF'][value_filter], catalog_3DHST.redshift['z_LEGAC'][value_filter], label=f"MOSDEF ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
    if field == 'COSMOS' and matches_FMOS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_FMOS'] > 0.0) & (catalog_3DHST.redshift['z_FMOS_flag'] < 8) & (catalog_3DHST.redshift['z_LEGAC'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_FMOS'][value_filter], catalog_3DHST.redshift['z_LEGAC'][value_filter], label=f"FMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3)
    if (field == 'COSMOS' or field == 'GOODS-S' or field == 'UDS') and matches_KMOS3D.count > 0:
        value_filter = (catalog_3DHST.redshift['z_KMOS3D'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D_flag'] < 8) & (catalog_3DHST.redshift['z_LEGAC'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_KMOS3D'][value_filter], catalog_3DHST.redshift['z_LEGAC'][value_filter], label=f"KMOS3D ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3)
    if field == 'COSMOS' and matches_C3R2.count > 0:
        value_filter = (catalog_3DHST.redshift['z_C3R2'] > 0.0) & (catalog_3DHST.redshift['z_C3R2_flag'] < 8) & (catalog_3DHST.redshift['z_LEGAC'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_C3R2'][value_filter], catalog_3DHST.redshift['z_LEGAC'][value_filter], label=f"C3R2 ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour2)
    ax.set_xlabel('$z_{spec}$ [Other]')
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_LEGAC.catalog}]")
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)
    ax.legend(loc='lower right')

    del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_LEGAC' in locals():
    catalog.close(catalog_LEGAC)
    del catalog_LEGAC, catalog_params_LEGAC

### Cross-Match HELP z-Spec Catalog (v2.8 Public)

In [ ]:
if field == 'AEGIS' or field == 'COSMOS':
    catalog_params_HELP = catalog.get_params('HELP', field)
    catalog_HELP = catalog.read(catalog_params_HELP)
    print(f"{catalog_HELP.catalog}: {catalog_HELP.count} sources")

In [ ]:
if field == 'AEGIS' or field == 'COSMOS':
    matches_HELP = match.cross_match_catalogs_by_ra_dec(catalog_3DHST, catalog_HELP)
    print(f"Number of Matches: {matches_HELP.count}")

In [ ]:
if field == 'AEGIS' or field == 'COSMOS':
    match.append_cross_matched_data(matches_HELP, catalog_3DHST, catalog_HELP)

In [ ]:
if (field == 'AEGIS' or field == 'COSMOS') and matches_HELP.count > 0:
    value_filter       = (catalog_3DHST.idmap['id']  < 1e7) & (catalog_3DHST.idmap['HELP'] >= 0)
    other_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['HELP'] >= 0) & (catalog_3DHST.idmap['HELP_dist'] > 0.0)
    multi_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['HELP'] >= 0) & (catalog_3DHST.idmap['HELP_dist'] == 0.0) & (catalog_3DHST.idmap['HELP_num'] > 1)
    dup_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['HELP'] >= 0) & (catalog_3DHST.idmap['HELP_dist'] == 0.0) & (catalog_3DHST.idmap['HELP_num'] == 1)
    non_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['HELP'] >= 0) & (catalog_3DHST.idmap['HELP_dist'] == 0.0) & (catalog_3DHST.idmap['HELP_num'] == 0)

    num_matches = np.sum(value_filter)
    num_other_matches = np.sum(other_match_filter)
    num_multi_matches = np.sum(multi_match_filter)
    num_dup_matches = np.sum(dup_match_filter)
    num_non_matches = np.sum(non_match_filter)
    print(f"Number of {catalog_HELP.name} Redshifts:")
    print(f"    Matches      : {num_matches} (diff={matches_HELP.count - num_matches - num_other_matches})")
    print(f"  + Other matches: {num_other_matches}")
    print(f"  + Dup-matches  : {num_dup_matches}")
    print(f"  + Multi-matches: {num_multi_matches}")
    print(f"  + Non-matches  : {num_non_matches}")
    print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_HELP.count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

    zlim_deep = [0, 6]
    zlim = [0, 6]

    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_HELP'] > 0.0) & (catalog_3DHST.redshift['z_HELP_flag'] < 8)
    fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_HELP.catalog} vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_HELP'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel(f"$z_{{peak}}$ [{catalog_3DHST.catalog}]")
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_HELP.catalog}]")
    ax.set_xlim(zlim_deep)
    ax.set_ylim(zlim_deep)

    fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_HELP.catalog}")
    if field == 'COSMOS' and matches_ZCB.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCB'] > 0.0) & (catalog_3DHST.redshift['z_ZCB_flag'] < 8) & (catalog_3DHST.redshift['z_HELP'] > 0.0) & (catalog_3DHST.redshift['z_HELP_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCB'][value_filter], catalog_3DHST.redshift['z_HELP'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
    if field == 'COSMOS' and matches_ZCD.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCD'] > 0.0) & (catalog_3DHST.redshift['z_ZCD_flag'] < 8) & (catalog_3DHST.redshift['z_HELP'] > 0.0) & (catalog_3DHST.redshift['z_HELP_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCD'][value_filter], catalog_3DHST.redshift['z_HELP'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
    if (field == 'COSMOS' or field == 'GOODS-S') and matches_VUDS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_VUDS'] > 0.0) & (catalog_3DHST.redshift['z_VUDS_flag'] < 8) & (catalog_3DHST.redshift['z_HELP'] > 0.0) & (catalog_3DHST.redshift['z_HELP_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_VUDS'][value_filter], catalog_3DHST.redshift['z_HELP'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'COSMOS' or field == 'GOODS-N' or field == 'UDS') and matches_Casey.count > 0:
        value_filter = (catalog_3DHST.redshift['z_Casey'] > 0.0) & (catalog_3DHST.redshift['z_Casey_flag'] < 8) & (catalog_3DHST.redshift['z_HELP'] > 0.0) & (catalog_3DHST.redshift['z_HELP_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_Casey'][value_filter], catalog_3DHST.redshift['z_HELP'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
    if field == 'COSMOS' and matches_DEIMOS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_DEIMOS'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS_flag'] < 8) & (catalog_3DHST.redshift['z_HELP'] > 0.0) & (catalog_3DHST.redshift['z_HELP_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_DEIMOS'][value_filter], catalog_3DHST.redshift['z_HELP'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'AEGIS' or field == 'COSMOS' or field == 'GOODS-N' or field == 'GOODS-S' or field == 'UDS') and matches_MOSDEF.count > 0:
        value_filter = (catalog_3DHST.redshift['z_MOSDEF'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF_flag'] < 8) & (catalog_3DHST.redshift['z_HELP'] > 0.0) & (catalog_3DHST.redshift['z_HELP_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_MOSDEF'][value_filter], catalog_3DHST.redshift['z_HELP'][value_filter], label=f"MOSDEF ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
    if field == 'COSMOS' and matches_FMOS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_FMOS'] > 0.0) & (catalog_3DHST.redshift['z_FMOS_flag'] < 8) & (catalog_3DHST.redshift['z_HELP'] > 0.0) & (catalog_3DHST.redshift['z_HELP_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_FMOS'][value_filter], catalog_3DHST.redshift['z_HELP'][value_filter], label=f"FMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
    if (field == 'COSMOS' or field == 'GOODS-S' or field == 'UDS') and matches_KMOS3D.count > 0:
        value_filter = (catalog_3DHST.redshift['z_KMOS3D'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D_flag'] < 8) & (catalog_3DHST.redshift['z_HELP'] > 0.0) & (catalog_3DHST.redshift['z_HELP_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_KMOS3D'][value_filter], catalog_3DHST.redshift['z_HELP'][value_filter], label=f"KMOS3D ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3)
    if field == 'COSMOS' and matches_C3R2.count > 0:
        value_filter = (catalog_3DHST.redshift['z_C3R2'] > 0.0) & (catalog_3DHST.redshift['z_C3R2_flag'] < 8) & (catalog_3DHST.redshift['z_HELP'] > 0.0) & (catalog_3DHST.redshift['z_HELP_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_C3R2'][value_filter], catalog_3DHST.redshift['z_HELP'][value_filter], label=f"C3R2 ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour2)
    if field == 'COSMOS' and matches_LEGAC.count > 0:
        value_filter = (catalog_3DHST.redshift['z_LEGAC'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC_flag'] < 8) & (catalog_3DHST.redshift['z_HELP'] > 0.0) & (catalog_3DHST.redshift['z_HELP_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_LEGAC'][value_filter], catalog_3DHST.redshift['z_HELP'][value_filter], label=f"LEGAC ({np.sum(value_filter)})", s=5, marker='^', edgecolors='none', facecolors=plot_options.colour3)
    ax.set_xlabel('$z_{spec}$ [Other]')
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_HELP.catalog}]")
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)
    ax.legend(loc='lower right')

    del zlim_deep, zlim, fig, ax

In [ ]:
# TODO: Check matches based on catalog_HELP.sources['Z_SOURCE']

In [ ]:
if not keep_open and 'catalog_HELP' in locals():
    catalog.close(catalog_HELP)
    del catalog_HELP, catalog_params_HELP

### Cross-Match DESI (EDR, 2023-06-09)

In [ ]:
catalog_params_DESI = catalog.get_params('DESI', field)
catalog_DESI = catalog.read(catalog_params_DESI, region=catalog_region_3DHST, region_wcs=catalog_params_3DHST.wcs)
print(f"{catalog_DESI.catalog}: {catalog_DESI.count} sources")

In [ ]:
matches_DESI = match.cross_match_catalogs_by_ra_dec(catalog_3DHST, catalog_DESI)
print(f"Number of Matches: {matches_DESI.count}")

In [ ]:
match.append_cross_matched_data(matches_DESI, catalog_3DHST, catalog_DESI)

In [ ]:
if matches_DESI.count > 0:
    value_filter   = (catalog_3DHST.idmap['id']  < 1e7) & (catalog_3DHST.idmap['DESI'] >= 0)
    other_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['DESI'] >= 0) & (catalog_3DHST.idmap['DESI_dist'] > 0.0)
    multi_match_filter = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['DESI'] >= 0) & (catalog_3DHST.idmap['DESI_dist'] == 0.0) & (catalog_3DHST.idmap['DESI_num'] > 1)
    dup_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['DESI'] >= 0) & (catalog_3DHST.idmap['DESI_dist'] == 0.0) & (catalog_3DHST.idmap['DESI_num'] == 1)
    non_match_filter   = (catalog_3DHST.idmap['id'] >= 1e7) & (catalog_3DHST.idmap['DESI'] >= 0) & (catalog_3DHST.idmap['DESI_dist'] == 0.0) & (catalog_3DHST.idmap['DESI_num'] == 0)

    num_matches = np.sum(value_filter)
    num_other_matches = np.sum(other_match_filter)
    num_multi_matches = np.sum(multi_match_filter)
    num_dup_matches = np.sum(dup_match_filter)
    num_non_matches = np.sum(non_match_filter)
    print(f"Number of {catalog_DESI.name} Redshifts:")
    print(f"    Matches      : {num_matches} (diff={matches_DESI.count - num_matches - num_other_matches})")
    print(f"  + Other matches: {num_other_matches}")
    print(f"  + Dup-matches  : {num_dup_matches}")
    print(f"  + Multi-matches: {num_multi_matches}")
    print(f"  + Non-matches  : {num_non_matches}")
    print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_DESI.count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

    zlim_deep = [0, 3.5]
    zlim = [0, 2.0]

    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_DESI'] > 0.0) & (catalog_3DHST.redshift['z_DESI_flag'] < 8)
    fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_DESI.catalog} vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_DESI'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel(f"$z_{{peak}}$ [{catalog_3DHST.catalog}]")
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_DESI.catalog}]")
    ax.set_xlim(zlim_deep)
    ax.set_ylim(zlim_deep)

    fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_DESI.catalog}")
    if field == 'COSMOS' and matches_ZCB.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCB'] > 0.0) & (catalog_3DHST.redshift['z_ZCB_flag'] < 8) & (catalog_3DHST.redshift['z_DESI'] > 0.0) & (catalog_3DHST.redshift['z_DESI_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCB'][value_filter], catalog_3DHST.redshift['z_DESI'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
    if field == 'COSMOS' and matches_ZCD.count > 0:
        value_filter = (catalog_3DHST.redshift['z_ZCD'] > 0.0) & (catalog_3DHST.redshift['z_ZCD_flag'] < 8) & (catalog_3DHST.redshift['z_DESI'] > 0.0) & (catalog_3DHST.redshift['z_DESI_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_ZCD'][value_filter], catalog_3DHST.redshift['z_DESI'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
    if (field == 'COSMOS' or field == 'GOODS-S') and matches_VUDS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_VUDS'] > 0.0) & (catalog_3DHST.redshift['z_VUDS_flag'] < 8) & (catalog_3DHST.redshift['z_DESI'] > 0.0) & (catalog_3DHST.redshift['z_DESI_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_VUDS'][value_filter], catalog_3DHST.redshift['z_DESI'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'COSMOS' or field == 'GOODS-N' or field == 'UDS') and matches_Casey.count > 0:
        value_filter = (catalog_3DHST.redshift['z_Casey'] > 0.0) & (catalog_3DHST.redshift['z_Casey_flag'] < 8) & (catalog_3DHST.redshift['z_DESI'] > 0.0) & (catalog_3DHST.redshift['z_DESI_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_Casey'][value_filter], catalog_3DHST.redshift['z_DESI'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
    if field == 'COSMOS' and matches_DEIMOS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_DEIMOS'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS_flag'] < 8) & (catalog_3DHST.redshift['z_DESI'] > 0.0) & (catalog_3DHST.redshift['z_DESI_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_DEIMOS'][value_filter], catalog_3DHST.redshift['z_DESI'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
    if (field == 'AEGIS' or field == 'COSMOS' or field == 'GOODS-N' or field == 'GOODS-S' or field == 'UDS') and matches_MOSDEF.count > 0:
        value_filter = (catalog_3DHST.redshift['z_MOSDEF'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF_flag'] < 8) & (catalog_3DHST.redshift['z_DESI'] > 0.0) & (catalog_3DHST.redshift['z_DESI_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_MOSDEF'][value_filter], catalog_3DHST.redshift['z_DESI'][value_filter], label=f"MOSDEF ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
    if field == 'COSMOS' and matches_FMOS.count > 0:
        value_filter = (catalog_3DHST.redshift['z_FMOS'] > 0.0) & (catalog_3DHST.redshift['z_FMOS_flag'] < 8) & (catalog_3DHST.redshift['z_DESI'] > 0.0) & (catalog_3DHST.redshift['z_DESI_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_FMOS'][value_filter], catalog_3DHST.redshift['z_DESI'][value_filter], label=f"FMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
    if (field == 'COSMOS' or field == 'GOODS-S' or field == 'UDS') and matches_KMOS3D.count > 0:
        value_filter = (catalog_3DHST.redshift['z_KMOS3D'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D_flag'] < 8) & (catalog_3DHST.redshift['z_DESI'] > 0.0) & (catalog_3DHST.redshift['z_DESI_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_KMOS3D'][value_filter], catalog_3DHST.redshift['z_DESI'][value_filter], label=f"KMOS3D ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3)
    if field == 'COSMOS' and matches_C3R2.count > 0:
        value_filter = (catalog_3DHST.redshift['z_C3R2'] > 0.0) & (catalog_3DHST.redshift['z_C3R2_flag'] < 8) & (catalog_3DHST.redshift['z_DESI'] > 0.0) & (catalog_3DHST.redshift['z_DESI_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_C3R2'][value_filter], catalog_3DHST.redshift['z_DESI'][value_filter], label=f"C3R2 ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour2)
    if field == 'COSMOS' and matches_LEGAC.count > 0:
        value_filter = (catalog_3DHST.redshift['z_LEGAC'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC_flag'] < 8) & (catalog_3DHST.redshift['z_DESI'] > 0.0) & (catalog_3DHST.redshift['z_DESI_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_LEGAC'][value_filter], catalog_3DHST.redshift['z_DESI'][value_filter], label=f"LEGAC ({np.sum(value_filter)})", s=5, marker='^', edgecolors='none', facecolors=plot_options.colour3)
    if (field == 'COSMOS' or field == 'AEGIS') and matches_HELP.count > 0:
        value_filter = (catalog_3DHST.redshift['z_HELP'] > 0.0) & (catalog_3DHST.redshift['z_HELP_flag'] < 8) & (catalog_3DHST.redshift['z_DESI'] > 0.0) & (catalog_3DHST.redshift['z_DESI_flag'] < 8)
        ax.scatter(catalog_3DHST.redshift['z_HELP'][value_filter], catalog_3DHST.redshift['z_DESI'][value_filter], label=f"HELP ({np.sum(value_filter)})", s=5, marker='^', edgecolors='none', facecolors=plot_options.colour2)
    ax.set_xlabel('$z_{spec}$ [Other]')
    ax.set_ylabel(f"$z_{{spec}}$ [{catalog_DESI.catalog}]")
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)
    ax.legend(loc='lower right')

    del zlim_deep, zlim, fig, ax

In [ ]:
if matches_DESI.count > 0:
    z = catalog_DESI.sources['Z']
    dz = catalog_DESI.sources['ZERR']
    rdz = dz / (1 + z)

    fig, ax = plot.create_plot(plot_mode, title=f"DESI Redshift Error, N={len(z)}")
    ax.scatter(z, rdz, s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel('$z_{{spec}}$')
    ax.set_ylabel('$\sigma_z / (1+z)$')

    fig, ax = plot.create_plot(plot_mode, title=f"DESI Redshift Error, N={len(z)}")
    ax.hist(rdz*1e4, int(np.sqrt(len(rdz))), rwidth=0.95, color=plot_options.hist_colour)
    ax.set_xlabel('$\sigma_z / (1+z) \\times 10^{-4}$')
    ax.set_ylabel('Galaxies')
    ax.set_xlim([0, 2])

    del z, dz, rdz, fig, ax

In [ ]:
if not keep_open and 'catalog_DESI' in locals():
    catalog.close(catalog_DESI)
    del catalog_DESI, catalog_params_DESI

### Consolidate 3D-HST Catalog

In [ ]:
catalog.consolidate_best_data(catalog_3DHST)

### Summary

In [ ]:
value_filter = (catalog_3DHST.sources['star_flag'] != 1) & (catalog_3DHST.best['z_best'] > 0.0) & (catalog_3DHST.best['z_best_flag'] < 10)
sources, source_counts = np.unique(catalog_3DHST.best['z_best_source'][value_filter], return_counts=True)
source_ticks = np.arange(len(sources))
fig, ax = plot.create_plot(plot_mode, title=f"All $z_{{spec}}$, N={np.sum(value_filter)}")
b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
ax.set_xticks(source_ticks)
ax.set_xticklabels(sources, rotation=45, ha='right')
ax.set_xlabel('Source')
ax.set_ylabel('Galaxies')

print(f"All z-Specs: {np.sum(value_filter)}")

value_filter = (catalog_3DHST.sources['star_flag'] != 1) & (catalog_3DHST.best['z_best'] > 0.0) & (catalog_3DHST.best['z_best_flag'] < 8)
sources, source_counts = np.unique(catalog_3DHST.best['z_best_source'][value_filter], return_counts=True)
source_ticks = np.arange(len(sources))
fig, ax = plot.create_plot(plot_mode, title=f"Usable $z_{{spec}}$, N={np.sum(value_filter)}")
b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
ax.set_xticks(source_ticks)
ax.set_xticklabels(sources, rotation=45, ha='right')
ax.set_xlabel('Source')
ax.set_ylabel('Galaxies')

print(f"Usable z-Specs: {np.sum(value_filter)}")

value_filter = (catalog_3DHST.sources['star_flag'] != 1) & (catalog_3DHST.best['z_best'] > 0.0) & (catalog_3DHST.best['z_best_flag'] < 7)
sources, source_counts = np.unique(catalog_3DHST.best['z_best_source'][value_filter], return_counts=True)
source_ticks = np.arange(len(sources))
fig, ax = plot.create_plot(plot_mode, title=f"Best $z_{{spec}}$, N={np.sum(value_filter)}")
b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
ax.set_xticks(source_ticks)
ax.set_xticklabels(sources, rotation=45, ha='right')
ax.set_xlabel('Source')
ax.set_ylabel('Galaxies')

print(f"Best z-Specs: {np.sum(value_filter)}")

value_filter = (catalog_3DHST.sources['star_flag'] != 1) & (catalog_3DHST.best['z_best'] > 0.0) & (catalog_3DHST.best['z_best_flag'] < 7) & (catalog_3DHST.best['id'] < 1e7)
sources, source_counts = np.unique(catalog_3DHST.best['z_best_source'][value_filter], return_counts=True)
source_ticks = np.arange(len(sources))
fig, ax = plot.create_plot(plot_mode, title=f"Best Matched $z_{{spec}}$, N={np.sum(value_filter)}")
b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
ax.set_xticks(source_ticks)
ax.set_xticklabels(sources, rotation=45, ha='right')
ax.set_xlabel('Source')
ax.set_ylabel('Galaxies')

print(f"Best Matched z-Specs: {np.sum(value_filter)}")

zlim = [0, 6]
fig, ax = plot.create_plot(plot_mode, title=f"Best Matched $z_{{spec}}$, N={np.sum(value_filter)}")
if field == 'COSMOS' and matches_ZCB.count > 0:
    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_ZCB_flag'] < 7)
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_ZCB'][value_filter], label='ZCB', s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
if field == 'COSMOS' and matches_ZCD.count > 0:
    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_ZCD_flag'] < 7)
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_ZCD'][value_filter], label='ZCD', s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3, alpha=0.6)
if (field == 'COSMOS' or field == 'GOODS-S') and matches_VUDS.count > 0:
    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_VUDS_flag'] < 7)
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_VUDS'][value_filter], label='VUDS', s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5, alpha=0.6)
if (field == 'COSMOS' or field == 'GOODS-N' or field == 'UDS') and matches_Casey.count > 0:
    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_Casey_flag'] < 7)
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_Casey'][value_filter], label='Casey', s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6, alpha=0.6)
if field == 'COSMOS' and matches_DEIMOS.count > 0:
    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_DEIMOS_flag'] < 7)
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_DEIMOS'][value_filter], label='DEIMOS', s=5, marker='s', edgecolors='none', facecolors=plot_options.colour5, alpha=0.6)
if (field == 'AEGIS' or field == 'COSMOS' or field == 'GOODS-N' or field == 'GOODS-S' or field == 'UDS') and matches_MOSDEF.count > 0:
    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_MOSDEF_flag'] < 7)
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_MOSDEF'][value_filter], label='MOSDEF', s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4, alpha=0.6)
if field == 'COSMOS' and matches_FMOS.count > 0:
    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_FMOS_flag'] < 7)
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_FMOS'][value_filter], label='FMOS', s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3, alpha=0.6)
if (field == 'COSMOS' or field == 'GOODS-S' or field == 'UDS') and matches_KMOS3D.count > 0:
    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_KMOS3D_flag'] < 7)
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_KMOS3D'][value_filter], label='KMOS3D', s=5, marker='s', edgecolors='none', facecolors=plot_options.colour2, alpha=0.6)
if field == 'COSMOS' and matches_C3R2.count > 0:
    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_C3R2_flag'] < 7)
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_C3R2'][value_filter], label='C3R2', s=5, marker='^', edgecolors='none', facecolors=plot_options.colour3, alpha=0.6)
if field == 'COSMOS' and matches_LEGAC.count > 0:
    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_LEGAC_flag'] < 7)
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_LEGAC'][value_filter], label='LEGAC', s=5, marker='^', edgecolors='none', facecolors=plot_options.colour5, alpha=0.6)
if (field == 'COSMOS' or field == 'AEGIS') and matches_HELP.count > 0:
    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_HELP_flag'] < 7)
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_HELP'][value_filter], label='HELP', s=5, marker='^', edgecolors='none', facecolors=plot_options.colour5, alpha=0.6)
if matches_DESI.count > 0:
    value_filter = (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.redshift['z_DESI_flag'] < 7)
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.redshift['z_DESI'][value_filter], label='DESI', s=5, marker='^', edgecolors='none', facecolors=plot_options.colour6, alpha=0.6)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_3DHST.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del fig, ax, value_filter


In [ ]:
zlim = [0, 6]

value_filter = (catalog_3DHST.sources['star_flag'] != 1) & (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.best['z_best'] > 0.0) & (catalog_3DHST.best['z_best_flag'] < 7)

fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Best) vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.best['z_best'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel('$z_{peak}$')
ax.set_ylabel('$z_{best}$')
ax.set_xlim(zlim)
ax.set_ylim(zlim)

sources, source_counts = np.unique(catalog_3DHST.best['z_best_source'][value_filter], return_counts=True)
source_ticks = np.arange(len(sources))
fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Best), N={np.sum(value_filter)}")
b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
ax.set_xticks(source_ticks)
ax.set_xticklabels(sources, rotation=45, ha='right')
ax.set_xlabel('Source')
ax.set_ylabel('Galaxies')

value_filter = (catalog_3DHST.sources['star_flag'] != 1) & (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.best['z_best'] > 0.0) & (catalog_3DHST.best['z_best_flag'] > 2) & (catalog_3DHST.best['z_best_flag'] < 8)
if np.sum(value_filter) > 0:
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Other) vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.best['z_best'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel('$z_{peak}$')
    ax.set_ylabel('$z_{best}$')
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)

    sources, source_counts = np.unique(catalog_3DHST.best['z_best_source'][value_filter], return_counts=True)
    source_ticks = np.arange(len(sources))
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Other), N={np.sum(value_filter)}")
    b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
    ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
    ax.set_xticks(source_ticks, sources)
    ax.set_xticklabels(sources, rotation=45, ha='right')
    ax.set_xlabel('Source')
    ax.set_ylabel('Galaxies')

value_filter = (catalog_3DHST.sources['star_flag'] != 1) & (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.best['z_best'] > 0.0) & (catalog_3DHST.best['z_best_flag'] == 8)
if np.sum(value_filter) > 0:
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Inconsistent) vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.best['z_best'][value_filter], label='Inconsistent', s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel('$z_{peak}$')
    ax.set_ylabel('$z_{best}$')
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)

    sources, source_counts = np.unique(catalog_3DHST.best['z_best_source'][value_filter], return_counts=True)
    source_ticks = np.arange(len(sources))
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Inconsistent), N={np.sum(value_filter)}")
    b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
    ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
    ax.set_xticks(source_ticks, sources)
    ax.set_xticklabels(sources, rotation=45, ha='right')
    ax.set_xlabel('Source')
    ax.set_ylabel('Galaxies')

value_filter = (catalog_3DHST.sources['star_flag'] != 1) & (catalog_3DHST.redshift['z_best'] > 0.0) & (catalog_3DHST.best['z_best'] > 0.0) & (catalog_3DHST.best['z_best_flag'] == 9)
if np.sum(value_filter) > 0:
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Unreliable) vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.best['z_best'][value_filter], label='Unreliable', s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel('$z_{peak}$')
    ax.set_ylabel('$z_{best}$')
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)

    sources, source_counts = np.unique(catalog_3DHST.best['z_best_source'][value_filter], return_counts=True)
    source_ticks = np.arange(len(sources))
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Unreliable), N={np.sum(value_filter)}")
    b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
    ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
    ax.set_xticks(source_ticks, sources)
    ax.set_xticklabels(sources, rotation=45, ha='right')
    ax.set_xlabel('Source')
    ax.set_ylabel('Galaxies')

value_filter = (catalog_3DHST.sources['star_flag'] != 1) & (catalog_3DHST.redshift['z_best'] > 0.0) & ((catalog_3DHST.best['z_best'] == 0.0) | (catalog_3DHST.best['z_best_flag'] == 99))
if np.sum(value_filter) > 0:
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (None) vs {catalog_3DHST.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_3DHST.redshift['z_best'][value_filter], catalog_3DHST.best['z_best'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel('$z_{peak}$')
    ax.set_ylabel('$z_{best}$')
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)

del zlim, fig, ax, b, value_filter

### Compare Emission Line Flux

In [ ]:
catalog_3DHST.lines['f_Ha_3DHST']       = catalog_3DHST.lines['Ha_FLUX']
catalog_3DHST.lines['e_Ha_3DHST']       = catalog_3DHST.lines['Ha_FLUX_ERR']
catalog_3DHST.lines['flag_Ha_3DHST']    = 1

catalog_3DHST.lines['f_OIIIb_3DHST']    = catalog_3DHST.lines['OIII_FLUX']
catalog_3DHST.lines['e_OIIIb_3DHST']    = catalog_3DHST.lines['OIII_FLUX_ERR']
catalog_3DHST.lines['flag_OIIIb_3DHST'] = 1

catalog_3DHST.lines['f_Hb_3DHST']       = catalog_3DHST.lines['Hb_FLUX']
catalog_3DHST.lines['e_Hb_3DHST']       = catalog_3DHST.lines['Hb_FLUX_ERR']
catalog_3DHST.lines['flag_Hb_3DHST']    = 1

catalog_3DHST.lines['f_OIIb_3DHST']     = catalog_3DHST.lines['OII_FLUX']
catalog_3DHST.lines['e_OIIb_3DHST']     = catalog_3DHST.lines['OII_FLUX_ERR']
catalog_3DHST.lines['flag_OIIb_3DHST']  = 1

In [ ]:
line_names = ['Ha', 'OIIIb', 'Hb', 'OIIb']
line_labels = ['Ha+[NII]', '[OIII]', 'Hb', '[OII]']
line_other_names = [['NIIa', 'NIIb'], ['OIIIa'], [], ['OIIa']]
if field == 'COSMOS':
    line_sources = ['3DHST', 'MOSDEF', 'FMOS', 'KMOS3D', 'LEGAC', 'DESI']
    line_markers = ['o'    , 's'     , '^'   , '>'     , 'v'    , 'p'  ]
    line_sizes =   [15     , 15      , 25    , 25      , 25     , 25    ]
elif field != 'GOODS-S' and field != 'UDS':
    line_sources = ['3DHST', 'MOSDEF', 'DESI']
    line_markers = ['o'    , 's'     , 'p'   ]
    line_sizes =   [15     , 15      , 25    ]
else:
    line_sources = ['3DHST', 'MOSDEF', 'KMOS3D']
    line_markers = ['o'    , 's'     , '>'     ]
    line_sizes =   [15     , 15      , 25      ]

line_colours = plot_options.colours[2:7]

fluxLim = [1e-1, 1e3]

for i in np.arange(len(line_names)):
    fig, ax = plot.create_plot(plot_mode, title=f"Compare $f_{{{line_labels[i]}}}$")

    for j in np.arange(len(line_sources)):
        total_flux_j = np.zeros((len(catalog_3DHST.lines)))
        value_filter = catalog_3DHST.lines[f"flag_{line_names[i]}_{line_sources[j]}"] < 9
        total_flux_j[value_filter] = catalog_3DHST.lines[f"f_{line_names[i]}_{line_sources[j]}"][value_filter]

        for l in np.arange(len(line_other_names[i])):
            if table.has_field(catalog_3DHST.lines, f"flag_{line_other_names[i][l]}_{line_sources[j]}"):
                value_filter = catalog_3DHST.lines[f"flag_{line_other_names[i][l]}_{line_sources[j]}"] < 9
                total_flux_j[value_filter] += catalog_3DHST.lines[f"f_{line_other_names[i][l]}_{line_sources[j]}"][value_filter]

        for k in np.arange(j+1, len(line_sources)):
            total_flux_k = np.zeros((len(catalog_3DHST.lines)))
            value_filter = catalog_3DHST.lines[f"flag_{line_names[i]}_{line_sources[k]}"] < 9
            total_flux_k[value_filter] = catalog_3DHST.lines[f"f_{line_names[i]}_{line_sources[k]}"][value_filter]

            for l in np.arange(len(line_other_names[i])):
                value_filter = catalog_3DHST.lines[f"flag_{line_other_names[i][l]}_{line_sources[k]}"] < 9
                total_flux_k[value_filter] += catalog_3DHST.lines[f"f_{line_other_names[i][l]}_{line_sources[k]}"][value_filter]

            value_filter = (total_flux_j > 0) & (total_flux_k > 0)

            if np.sum(value_filter) == 0:
                continue

            ax.scatter(total_flux_k[value_filter], total_flux_j[value_filter], label=f"{line_sources[j]} vs {line_sources[k]} ({np.sum(value_filter)})", s=line_sizes[k], marker=line_markers[k], facecolors=line_colours[j], edgecolors='k', linewidth=0.25)

    ax.set_xlabel(f"$f_{{{line_labels[i]}}}$")
    ax.set_ylabel(f"$f_{{{line_labels[i]}}}$")
    ax.plot(fluxLim, fluxLim, linestyle='-', color=plot_options.colour7)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlim(fluxLim)
    ax.set_ylim(fluxLim)
    ax.legend(loc='lower right')

del line_sources, line_names, line_other_names, line_labels, line_markers, line_sizes, line_colours, i, j, k, l, fluxLim, fig, ax


In [ ]:
if table.has_field(catalog_3DHST.lines, 'f_Ha_3DHST'):
    catalog_3DHST.lines.remove_columns(['f_Ha_3DHST'    ,'e_Ha_3DHST'    ,'flag_Ha_3DHST'   ])
    catalog_3DHST.lines.remove_columns(['f_OIIIb_3DHST' ,'e_OIIIb_3DHST' ,'flag_OIIIb_3DHST'])
    catalog_3DHST.lines.remove_columns(['f_Hb_3DHST'    ,'e_Hb_3DHST'    ,'flag_Hb_3DHST'   ])
    catalog_3DHST.lines.remove_columns(['f_OIIb_3DHST'  ,'e_OIIb_3DHST'  ,'flag_OIIb_3DHST' ])

### Save Cross-Matched 3D-HST Catalog

In [ ]:
catalog_params_3DHSTPLUS = catalog.get_params('3D-HST-PLUS')

if do_save:
    catalog.save_matched_catalog(catalog_3DHST, catalog_params_3DHSTPLUS, save_format='pickel', new_name='3D-HST-PLUS')

if do_save_ascii:
    catalog.save_matched_catalog(catalog_3DHST, catalog_params_3DHSTPLUS, save_format='ascii', new_name='3D-HST-PLUS')

### Close Catalog

In [ ]:
if not keep_open and 'catalog_3DHST' in locals():
    catalog.close(catalog_3DHST)
    del catalog_3DHST, catalog_params_3DHST, catalog_params_3DHSTPLUS